# Day 10

| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.
S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

In [1]:
ex1 = """..F7.
.FJ|.
SJ.L7
|F--J
LJ..."""

In [2]:
d = open('day10.txt').read()

In [3]:
def find_start(grid):
    for i in range(1,len(grid)):
        if grid[i].find("S") > -1:
            return (i,grid[i].find("S"))

In [4]:
# entrance point -> next pipe entrance point
pipe_dict = {
    '|' : {'t' : 't', 'b' : 'b'},
    '-' : {'l' : 'l', 'r' : 'r'},
    'L' : {'t' : 'l', 'r' : 'b'},
    'J' : {'t' : 'r', 'l' : 'b'},
    '7' : {'l' : 't', 'b' : 'r'},
    'F' : {'b' : 'l', 'r' : 't'}   
 }

pipe_dict['|']['b']

'b'

In [5]:
# if next pipes entrance point is top, this is the relative coord of the next pipe
dir_to_coord = {
    'b' : (-1,0),
    't' : (1,0),
    'r' : (0,-1), 
    'l' : (0,1) 
}


In [6]:
def find_adjacent_indexes(pos, width, height):
    adjacent = dict()
    x,y = pos
    # Check each of the four adjacent positions
    if x > 0:  # pipe with right entrance
        adjacent['r'] = (x-1, y)
    if x < width - 1:  # pipe with left entrance
        adjacent['l'] =  (x+1, y)
    if y > 0:  # pipe with bottom entrance (above)
        adjacent['b'] =  (x, y-1)
    if y < height - 1:  # pipe with top entrance (below)
        adjacent['t'] = (x, y+1)

    return adjacent

In [7]:
def first_position(grid):
    start_pos = find_start(grid)
    adj = find_adjacent_indexes(start_pos,len(grid[0]),len(grid))
    for i in ['l','r','t','b']:
        if adj.get(i):
            new_pipe = grid[adj[i][0]][adj[i][1]]
            if pipe_dict.get(new_pipe):
                if pipe_dict[new_pipe].get(i):
                    if pipe_dict[new_pipe][i] == 'l':
                        move = 'r'
                    if pipe_dict[new_pipe][i] == 'r':
                        move = 'l'
                    if pipe_dict[new_pipe][i] == 't':
                        move = 'b'
                    if pipe_dict[new_pipe][i] == 'b':
                        move = 't'
    return tuple(a + b for a, b in zip(start_pos, dir_to_coord[move])) , move    

In [9]:
def move_direction(current_pos, next_entrance, grid):
    current_pipe = grid[current_pos[0]][current_pos[1]]
    entrance_point = pipe_dict[current_pipe][next_entrance] #next pipes entrance point
    move_coord = dir_to_coord[entrance_point]
    new_pos = tuple(a+b for a,b in zip(current_pos,move_coord))
    print(new_pos)
    
    if grid[new_pos[0]][new_pos[1]] == 'S':
        return 1
    else:
        return move_direction(new_pos, entrance_point, grid = grid) + 1
    

In [11]:
# take a current position and next entrance, and return coord and entrance next position 
def move_direction(current_pos, next_entrance, grid):
    current_pipe = grid[current_pos[0]][current_pos[1]]
    entrance_point = pipe_dict[current_pipe][next_entrance] #next pipes entrance point
    move_coord = dir_to_coord[entrance_point]
    new_pos = tuple(a+b for a,b in zip(current_pos,move_coord))
    return new_pos, entrance_point
    

In [18]:
def p1(inp):
    grid = inp.splitlines()
    current_pos, next_entrance = first_position(grid)
    steps = 1 # one to get off the S
    while True:
        current_pipe = grid[current_pos[0]][current_pos[1]]
        if current_pipe == "S":
            return int(steps/2)
        else:
            current_pos, next_entrance = move_direction(current_pos, next_entrance, grid)
            steps = steps +1

In [19]:
p1(d)

6942

## Part 2

In [22]:
# modify part 1 to record the coords of the path
def p2(inp):
    grid = inp.splitlines()
    current_pos, next_entrance = first_position(grid)
    loop_coords = [find_start(grid), current_pos]
    steps = 1 # one to get off the S
    while True:
        current_pipe = grid[current_pos[0]][current_pos[1]]
        if current_pipe == "S":
            return steps/2 , loop_coords
        else:
            current_pos, next_entrance = move_direction(current_pos, next_entrance, grid)
            loop_coords.append(current_pos)
            steps = steps +1

In [23]:
ans, loop_coords = p2(d)

In [28]:
grid = d.splitlines()
new_grid = [list(s) for s in grid]

for i in range(len(new_grid)):
    for j in range(len(new_grid)):
        if (i,j) not in loop_coords:
            new_grid[i][j] = '*'

In [30]:
string_representation = '\n'.join([''.join(inner_list) for inner_list in new_grid])

print(string_representation)

********************************************************************************************************************************************
********************************************************************************************************************************************
********************************************************************************************************************************************
********************************************************************************************************************************************
************************************************************************************F7**************************F7**************************
************************************************************************************|L-7**********************F7||**************************
************************************************************************************L-7L7*********************|LJ|**************************
*************

In [34]:
count = 0
 
for row in new_grid:
    inside = False
    pos = 0
    while pos < len(row):
        if row[pos] == '*' and inside:
            count += 1
            pos += 1
        # if you cross | you go from out to in or vice versa
        elif row[pos] == '|':
            inside = not inside
            pos += 1
        elif row[pos] in ['-', '*']: # these don't tell us whether we've crossed a boundary so move along
            pos += 1
        else:
        # handling corner pipes e.g. L----7 or F---------J which indicate a boundary crossing.
        # passing, e.g., F-----7 does not indicate a crossing 
            turn_start = row[pos]
            pos += 1
            while row[pos] == '-':
                pos += 1
            turn_end = row[pos]
            if turn_start + turn_end in ['L7', 'FJ']:
                inside = not inside
            pos += 1
        
print(count)

297
